In [45]:
import sys

import matplotlib.pyplot as plt

%matplotlib notebook
%matplotlib notebook
## save variables
import pickle
## folder names
from glob import glob
## standard libraries
import numpy as np

##
#from sklearn.decomposition import PCA
#from sklearn.preprocessing import normalize


#!{sys.executable} -m pip install tensorflow-gpu --user
#!{sys.executable} -m pip install keras --user
import tensorflow as tf
from keras import backend as k

import keras
from keras.optimizers import SGD, Adam
#!{sys.executable} -m pip install hyperas --user
#!{sys.executable} -m pip install networkx==1.11 --user

from hyperas import optim
from hyperas.distributions import choice, uniform, conditional
from hyperopt import Trials, STATUS_OK, tpe

import os.path
import datetime

from dependencies import models
from dependencies import functions

from dependencies.convnet_drawer.convnet_drawer import Model
from dependencies.convnet_drawer.convnet_drawer import Conv2D
from dependencies.convnet_drawer.convnet_drawer import MaxPooling2D as MaxPooling2D_drawer
from dependencies.convnet_drawer.convnet_drawer import Flatten as Flatten_drawer
from dependencies.convnet_drawer.convnet_drawer import Dense as Dense_drawer
from dependencies.convnet_drawer.matplotlib_util import save_model_to_file
from dependencies.convnet_drawer.keras_util import convert_drawer_model

# Import mfcc data

In [41]:
#choose which dictionary to use
choice =     'mfcc'#'logfilter' #
useDelta =  True
#retrieving of used values for the computation of mfcc
with open('variables/mfccValues.pkl', 'rb') as f:  
    values = pickle.load(f)
selected = 0
if choice == 'mfcc': 
    
    
    #name format of the selected data
    if useDelta:
        name = 'variables/mfccDict_DD[nC='+str(values[selected][0])+' wL='+str(values[selected][2])+' wS='+str(values[selected][3])+'].pkl'
    else:
        name = 'variables/mfccDict[nC='+str(values[selected][0])+' wL='+str(values[selected][2])+' wS='+str(values[selected][3])+'].pkl'
    #loading in usedDict of the mfcc dict
    #name = 'variables/mfcc_delta_deltadelta_PCA.pkl'
    with open(name, 'rb') as f: 
        usedDict = pickle.load(f)
    print('Loaded '+name)

elif choice == 'logfilter':
    if useDelta:
        name = 'variables/logfiltDict_DD[nF='+str(values[selected][1])+' wL='+str(values[selected][2])+' wS='+str(values[selected][3])+'].pkl'
    else:
        name = 'variables/logfiltDict[nF='+str(values[selected][1])+' wL='+str(values[selected][2])+' wS='+str(values[selected][3])+'].pkl'
    #name = 'variables/pcalogDict.pkl'
    #saving in usedDict of the logfilter dict
    with open(name, 'rb') as f:  
        usedDict = pickle.load(f)
    print('Loaded '+name)

Loaded variables/mfccDict_DD[nC=14 wL=0.025 wS=0.01].pkl


# Acquiring and scaling data

In [42]:
#core words of the dataset
coreKey = ["yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go", "zero",
           "one", "two", "three", "four", "five", "six", "seven", "eight", "nine"]

#split of the core set
numbers = ['one', 'two', 'three','four','five','six','seven','eight','nine', "zero"]

words = ["yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go"]

test = ["yes", "up", "down", "left"]#, "right", "on", "off", "stop", "go", "zero",
          # "one", "two", "three", "four", "five", "six", "seven", "eight", "nine"]

#selecting the subset of words
used = words

used.append('silence')

unknown = list(usedDict.keys())
for key in used:
    try:
        unknown.remove(key)
    except:
        print(key, ' not in used')




In [43]:
%%time
#divding between train and test with also scaling data
functions.train_test_creator(
    {k: usedDict[k] for k in usedDict.keys() & used },
    {k: usedDict[k] for k in usedDict.keys() & unknown },
    with_unknown = False,
    scalerType = 'robust',
    depth = (len(usedDict[words[0]].shape)-3)*2 + 1 )
print()

{'copy': True, 'quantile_range': (25.0, 75.0), 'with_centering': True, 'with_scaling': True}
{'copy': True, 'quantile_range': (25.0, 75.0), 'with_centering': True, 'with_scaling': True}
{'copy': True, 'quantile_range': (25.0, 75.0), 'with_centering': True, 'with_scaling': True}

Wall time: 18.5 s


In [44]:
%reset -f
from IPython.display import Javascript
Javascript("Jupyter.notebook.execute_cells([0])")

<IPython.core.display.Javascript object>

In [46]:
x_train, y_train, x_test, y_test = functions.data()
with open('variables/labelList.pkl', 'rb') as f: 
        labelList = pickle.load(f)

# Fitting

In [47]:
# TensorFlow wizardry
config = tf.ConfigProto()
 
# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True
 
# Only allow a total 0.9 of the GPU memory to be allocated
config.gpu_options.per_process_gpu_memory_fraction = 0.8
 
# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

epoch = 40
epochSGD = 20

dest_directory = 'model_backup/'
if not os.path.exists(dest_directory):
      os.makedirs(dest_directory)

tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)

In [ ]:
cnn = models.model1(x_train,y_train, depth = 3)
print("Model1")
cnn.summary()

cnn = models.model2(x_train,y_train, depth = 3)
print("Model2")
cnn.summary()

cnn = models.model3(x_train,y_train, depth = 3)
print("Model3")
cnn.summary()

cnn = models.tinyDarknet(x_train,y_train, depth = 3)
print("tiny darknet")
cnn.summary()

In [ ]:
%%time

print("MODEL1")
cnn = models.model1(x_train,y_train, depth = x_train.shape[3])

cnn.summary()


compiledAdam = cnn.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])
fittedAdam = cnn.fit(x_train, y_train,
                     epochs=epoch,
                     validation_data=(x_test, y_test),
                     batch_size=round(x_train.shape[0]/300),
                     shuffle=True,
                     callbacks = [tbCallBack])

sgd = SGD(lr=0.01, decay=0.0005, momentum=0.9, nesterov=True)
compiledSGD = cnn.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=['accuracy'])

fittedSGD = cnn.fit(x_train, y_train, 
                epochs=epochSGD, 
                validation_data=(x_test, y_test), 
                batch_size=round(x_train.shape[0]/300), 
                shuffle=True,
                callbacks = [tbCallBack])

fig, ax = plt.subplots()
x = range(epoch)
for key in fittedAdam.history:
    ax.plot(x,fittedAdam.history[key],label=key)
legend = ax.legend(loc='upper center', shadow=True)
frame = legend.get_frame()
frame.set_facecolor('0.90')
for label in legend.get_texts():
    label.set_fontsize('large')

for label in legend.get_lines():
    label.set_linewidth(1.5)  # the legend line width

fig, ax = plt.subplots()
x = range(epochSGD)
for key in fittedSGD.history:
    ax.plot(x,fittedSGD.history[key],label=key)
legend = ax.legend(loc='upper center', shadow=True)
frame = legend.get_frame()
frame.set_facecolor('0.90')
for label in legend.get_texts():
    label.set_fontsize('large')

for label in legend.get_lines():
    label.set_linewidth(1.5)  # the legend line width


plt.show()

name = 'cnn1.bak'
cnn.save(dest_directory + name)

In [ ]:
%%time
print("\nMODEL3")
cnn = models.model3(x_train,y_train, depth = 3, basedim = 32)

cnn.summary()


compiledAdam = cnn.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])
fittedAdam = cnn.fit(x_train, y_train,
                     epochs=epoch,
                     validation_data=(x_test, y_test),
                     batch_size=round(x_train.shape[0]/200),
                     shuffle=True,
                     callbacks = [tbCallBack])

sgd = SGD(lr=0.01, decay=0.0005, momentum=0.9, nesterov=True)
compiledSGD = cnn.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=['accuracy'])

fittedSGD = cnn.fit(x_train, y_train, 
                epochs=epochSGD, 
                validation_data=(x_test, y_test), 
                batch_size=round(x_train.shape[0]/200), 
                shuffle=True,
                callbacks = [tbCallBack])

fig, ax = plt.subplots()
x = range(epoch)
for key in fittedAdam.history:
    ax.plot(x,fittedAdam.history[key],label=key)
legend = ax.legend(loc='upper center', shadow=True)
frame = legend.get_frame()
frame.set_facecolor('0.90')
for label in legend.get_texts():
    label.set_fontsize('large')

for label in legend.get_lines():
    label.set_linewidth(1.5)  # the legend line width

fig, ax = plt.subplots()
x = range(epochSGD)
for key in fittedSGD.history:
    ax.plot(x,fittedSGD.history[key],label=key)
legend = ax.legend(loc='upper center', shadow=True)
frame = legend.get_frame()
frame.set_facecolor('0.90')
for label in legend.get_texts():
    label.set_fontsize('large')

for label in legend.get_lines():
    label.set_linewidth(1.5)  # the legend line width
    
plt.show()

name = 'cnn3.bak'
cnn.save(dest_directory + name)

In [ ]:
%%time
print("\nTINYDARKNET")
cnn = models.tinyDarknet(x_train,y_train, depth = 3, dropout = 0.05)
#cnn.summary()


compiledAdam = cnn.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])
fittedAdam = cnn.fit(x_train, y_train,
                     epochs=epoch,
                     validation_data=(x_test, y_test),
                     batch_size=round(x_train.shape[0]/200),
                     shuffle=True,
                     callbacks = [tbCallBack])

sgd = SGD(lr=0.01, decay=0.0005, momentum=0.9, nesterov=True)
compiledSGD = cnn.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=['accuracy'])

fittedSGD = cnn.fit(x_train, y_train, 
                epochs=epochSGD, 
                validation_data=(x_test, y_test), 
                batch_size=round(x_train.shape[0]/200), 
                shuffle=True,
                callbacks = [tbCallBack])

fig, ax = plt.subplots()
x = range(epoch)
for key in fittedAdam.history:
    ax.plot(x,fittedAdam.history[key],label=key)
legend = ax.legend(loc='upper center', shadow=True)
frame = legend.get_frame()
frame.set_facecolor('0.90')
for label in legend.get_texts():
    label.set_fontsize('large')

for label in legend.get_lines():
    label.set_linewidth(1.5)  # the legend line width

fig, ax = plt.subplots()
x = range(epochSGD)
for key in fittedSGD.history:
    ax.plot(x,fittedSGD.history[key],label=key)
legend = ax.legend(loc='upper center', shadow=True)
frame = legend.get_frame()
frame.set_facecolor('0.90')
for label in legend.get_texts():
    label.set_fontsize('large')

for label in legend.get_lines():
    label.set_linewidth(1.5)  # the legend line width
    
plt.show()

name = 'cnn4.bak'
cnn.save(dest_directory + name)

In [ ]:
#!tensorboard --logdir Graph/

# Inception

In [48]:
from keras.layers import Input
from keras.layers import Flatten, Dense, Activation, BatchNormalization, Dropout, SpatialDropout2D
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
from keras import regularizers
from keras.models import Model


def cBN(inputLayer, filt = 64, size = (1,1), padding = 'same', activation = 'relu', regu = 0.0, dropout = 0.05, strides = (1,1)):
    cbn = Conv2D(filt, size, padding=padding, use_bias=False, kernel_regularizer=regularizers.l2(regu), strides = (1,1))(inputLayer)
    cbn = Activation(activation)(cbn)
    #cbn = BatchNormalization(epsilon=1e-05, momentum=0.1, axis=-1)(cbn)
    return cbn

def inception(inputLayer, filt = 64):
    tower_1 = cBN(inputLayer, filt = filt)   
    tower_1 = cBN(tower_1, size = (3,3), filt = filt)

    tower_2 = cBN(inputLayer, filt = filt)
    tower_2 = cBN(tower_2, size = (5,5), filt = filt)

    tower_3 = MaxPooling2D((3,2), strides=(1,1), padding='same')(inputLayer)
    tower_3 = cBN(tower_3, filt = filt)
    
    #tower_4 = cBN(inputLayer)

    output = keras.layers.concatenate([tower_1, tower_2, tower_3], axis = 3)
    return output

## Single output

In [ ]:
input_img = Input(name = 'input', shape = (x_train.shape[1], x_train.shape[2], x_train.shape[3]))

output = inception(input_img, filt = 20)
output = inception(input_img, filt = 30)

output = MaxPooling2D(pool_size=(3,2), padding='same')(output)
output = SpatialDropout2D(0.15)(output)

output = inception(output,filt = 30)
output = inception(output, filt = 40)

output = MaxPooling2D(pool_size=(3,2), padding='same')(output)
output = SpatialDropout2D(0.15)(output)

output = inception(output, filt = 40)
output = inception(output, filt = 60)

output = MaxPooling2D(pool_size=(3,2), padding='same')(output)
output = SpatialDropout2D(0.15)(output)

output = inception(output, filt = 60)
output = inception(output, filt = 100)

output = GlobalAveragePooling2D()(output)
#output = Dropout(0.2)(output)

output = Dense(100)(output)
#output = BatchNormalization(epsilon=1e-05, momentum=0.1)(output)
output = Activation('relu')(output)
output = Dropout(0.4)(output)
output = Dense(y_train.shape[1], name = 'output2', activation='softmax')(output)

In [ ]:
cnn = Model(inputs = input_img, outputs = output)
validation_data=({'input': x_test}, {'output2': y_test})
cnn.summary()

In [ ]:
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001/epoch, amsgrad=True)
compiledAdam = cnn.compile(loss="categorical_crossentropy", optimizer=adam, metrics=['categorical_accuracy'], loss_weights={'output2' : 1.})
fittedAdam = cnn.fit(x_train,  y_train,
                     epochs=epoch,
                     batch_size=round(x_train.shape[0]/400),
                     shuffle=True,
                     validation_data=validation_data, 
                     callbacks = [tbCallBack])

In [ ]:
sgd = SGD(lr=0.001, decay=0.001/epochSGD, momentum=0.9, nesterov=True)
compiledSGD = cnn.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=['accuracy'], loss_weights={'output2' : 1.})
fittedSGD = cnn.fit(x_train, y_train,
                     epochs=epochSGD,
                     batch_size=round(x_train.shape[0]/400),
                     validation_data=validation_data,
                     shuffle=True,
                     callbacks = [tbCallBack])

## Multiple input

In [91]:
def lin_inception(inputLayer, filt = 64, base = 3):
    tower_1 = cBN(inputLayer, filt = filt)   
    tower_1 = cBN(tower_1, size = (base,1), filt = filt)

    tower_2 = cBN(inputLayer, filt = filt)
    tower_2 = cBN(tower_2, size = (base+1,1), filt = filt)

    tower_3 = cBN(inputLayer, filt = filt)
    tower_3 = cBN(tower_3, filt = filt, size = (base+2,1))
    
    output = keras.layers.concatenate([tower_1, tower_2, tower_3], axis = 3)
    return output
    
def singleInput(x_train, name):
    single_input = Input(name = name, shape = (x_train.shape[1], x_train.shape[2], 1))
    output = Conv2D(120, (3,round(x_train.shape[2])))(single_input)
    output = lin_inception(output, filt = 80, base = 3)
    return single_input, output
def firstConcat(x_train):
    first, first_output = singleInput(x_train,'first')
    second, second_output = singleInput(x_train,'second')
    third, third_output = singleInput(x_train,'third')
    concat = keras.layers.concatenate([first_output, second_output, third_output], axis = 3)
    return first, second, third, concat

first_input, second_input, third_input, concat = firstConcat(x_train)
output = MaxPooling2D(pool_size=(3,1), padding='same')(concat)
output = SpatialDropout2D(0.15)(output)

output = lin_inception(output, filt = 80, base = 2)
output = lin_inception(output, filt = 80, base = 3)
'''
output = MaxPooling2D(pool_size=(3,1), padding='same')(output)
output = SpatialDropout2D(0.15)(output)

output = lin_inception(output, filt = 50, base = 4)
output = lin_inception(output, filt = 50, base = 5)

output = MaxPooling2D(pool_size=(3,1), padding='same')(output)
output = SpatialDropout2D(0.15)(output)

output = lin_inception(output, filt = 50, base = 4)
output = lin_inception(output, filt = 50, base = 5)

output = GlobalAveragePooling2D()(output)
'''
output = AveragePooling2D((11,1))(output)

#output = Flatten()(output)
output = Dense(100)(output)
output = Activation('relu')(output)
output = Dropout(0.6)(output)

output = Dense(y_train.shape[1], name = 'output', activation='softmax')(output)



In [92]:
cnn = Model(inputs = [first_input, second_input, third_input], outputs = output)
validation_data=({'first': x_test[:,:,:,0, np.newaxis], 'second': x_test[:,:,:,1, np.newaxis], 'third': x_test[:,:,:,2, np.newaxis]}, {'output': y_test})

cnn.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
first (InputLayer)              (None, 99, 14, 1)    0                                            
__________________________________________________________________________________________________
second (InputLayer)             (None, 99, 14, 1)    0                                            
__________________________________________________________________________________________________
third (InputLayer)              (None, 99, 14, 1)    0                                            
__________________________________________________________________________________________________
conv2d_818 (Conv2D)             (None, 97, 1, 120)   5160        first[0][0]                      
__________________________________________________________________________________________________
conv2d_825

In [83]:
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001/epoch, amsgrad=True)
compiledAdam = cnn.compile(loss="categorical_crossentropy", optimizer=adam, metrics=['categorical_accuracy'], loss_weights={'output' : 1.})
fittedAdam = cnn.fit([x_train[:,:,:,0, np.newaxis], x_train[:,:,:,1, np.newaxis], x_train[:,:,:,2, np.newaxis]],  y_train,
                     epochs=epoch,
                     batch_size=round(x_train.shape[0]/400),
                     shuffle=True,
                     validation_data=validation_data, 
                     callbacks = [tbCallBack])


Train on 34836 samples, validate on 8710 samples
Epoch 1/40
34836/34836 [==============================] - 14s 410us/step - loss: 1.0319 - categorical_accuracy: 0.6530 - val_loss: 0.3518 - val_categorical_accuracy: 0.8874
Epoch 2/40
34836/34836 [==============================] - 11s 325us/step - loss: 0.3374 - categorical_accuracy: 0.8950 - val_loss: 0.2698 - val_categorical_accuracy: 0.9189
Epoch 3/40
34836/34836 [==============================] - 11s 324us/step - loss: 0.2423 - categorical_accuracy: 0.9269 - val_loss: 0.2179 - val_categorical_accuracy: 0.9341
Epoch 4/40
34836/34836 [==============================] - 11s 324us/step - loss: 0.2007 - categorical_accuracy: 0.9391 - val_loss: 0.2472 - val_categorical_accuracy: 0.9281
Epoch 5/40
34836/34836 [==============================] - 11s 324us/step - loss: 0.1689 - categorical_accuracy: 0.9481 - val_loss: 0.1841 - val_categorical_accuracy: 0.9421
Epoch 6/40
34836/34836 [==============================] - 11s 324us/step - loss: 0.158

## Multiple output

In [ ]:
def extraClassifier(name, inputLayer, outputShape):
    output0 = GlobalAveragePooling2D()(inputLayer)
    output0 = Dense(100, activation = 'relu')(output0)
    output0 = Dropout(0.7)(output0)
    output0 = Dense( outputShape, name = name, activation='softmax')(output0)
    return output0

In [ ]:
input_img = Input(name = 'input', shape = (x_train.shape[1], x_train.shape[2], x_train.shape[3]))

output = cBN(input_img)
output = cBN(output, size = (3,3))
output = cBN(output)
output = cBN(output, size = (3,3))

output = MaxPooling2D(pool_size=(3,2), padding='same')(output)

output = inception(output)
output0 = extraClassifier('output0', output, y_train.shape[1])
output = inception(output)

output = MaxPooling2D(pool_size=(3,2), padding='same')(output)

output = inception(output)
output1 = extraClassifier('output1',output, y_train.shape[1])
output = inception(output)

output = cBN(output)
output = GlobalAveragePooling2D()(output)

#output = Dense(90, activation = 'relu')(output)
output = Dropout(0.5)(output)
output2 = Dense(y_train.shape[1], name = 'output2', activation='softmax')(output)

In [ ]:
cnn = Model(inputs = input_img, outputs = [output2, output1, output0])
validation_data=({'input': x_test}, {'output0': y_test, 'output1': y_test, 'output2': y_test})
cnn.summary()

In [ ]:
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
compiledAdam = cnn.compile(loss="categorical_crossentropy", optimizer=adam, metrics=['accuracy'], loss_weights={'output0': 1., 'output1': 1., 'output2' : 1.})
fittedAdam = cnn.fit(x_train, [y_train, y_train, y_train],
                     epochs=epoch,
                     batch_size=round(x_train.shape[0]/400),
                     shuffle=True,
                     validation_data=validation_data, 
                     callbacks = [tbCallBack])

In [ ]:
sgd = SGD(lr=0.005, decay=0.0005, momentum=0.9, nesterov=True)
compiledSGD = cnn.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=['accuracy'], loss_weights={'output0': 0.4, 'output1': 0.7, 'output2' : 1.})
fittedSGD = cnn.fit(x_train, [y_train, y_train, y_train],
                     epochs=epochSGD,
                     batch_size=round(x_train.shape[0]/400),
                     validation_data=validation_data,
                     shuffle=True,
                     callbacks = [tbCallBack])

# Score

In [ ]:
#find prediction with test data
preds = cnn.predict(x_test)
#print(list(used))

In [ ]:
#plot confusion matrix
functions.plotConfusionMatrix(preds[0],y_test,list(used))
precision = cnn.evaluate(x_test,[y_test,y_test,y_test])
#print ("Precision: ", round(precision*100,2),"%")

In [ ]:
precision

In [ ]:
fig, ax = plt.subplots()
x = range(epoch)
for key in fittedAdam.history:
    ax.plot(x,fittedAdam.history[key],label=key)
legend = ax.legend(loc='upper center', shadow=True)
frame = legend.get_frame()
frame.set_facecolor('0.90')
for label in legend.get_texts():
    label.set_fontsize('large')

for label in legend.get_lines():
    label.set_linewidth(1.5)  # the legend line width

plt.show()
    

# Output of conv2D

In [ ]:
cnn_copy = Sequential()
cnn_copy.add(cnn.layers[0])
result = cnn_copy.predict(x_test)


In [ ]:
plt.figure()
plt.imshow(result[0,:,:,0])

# Save Model

In [ ]:
dest_directory = 'model_backup/'
if not os.path.exists(dest_directory):
      os.makedirs(dest_directory)
name = 'cnn.bak'
cnn.save(dest_directory + name)

#bak = load_model(dest_directory + name)


In [ ]:
model = Model(input_shape=(x_train.shape[1], x_train.shape[2],1))
model.add(Conv2D(100, (4,4),  strides = (1,1), padding="valid"))
model.add(Conv2D(100, (4,2),  strides = (1,1), padding="valid"))
model.add(MaxPooling2D_drawer(pool_size=(3,3)))
model.add(Conv2D(128, (4,2),  strides = (1,1), padding="valid"))
model.add(Conv2D(128, (5,2),  strides = (1,1), padding="valid"))
model.add(MaxPooling2D_drawer(pool_size=(4,1)))
model.add(Flatten_drawer())
model.add(Dense_drawer(100))
model.add(Dense_drawer(y_train.shape[1]))

#save to pdf
save_model_to_file(model, "example.pdf")

# Looking for hyperparameters

In [ ]:
from keras.activations import softmax
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Flatten, Dropout, Convolution2D, MaxPooling2D, AveragePooling2D, BatchNormalization
trials = Trials()
best_run, best_model = optim.minimize(model=functions.create_model, 
                                      data=functions.data,
                                      algo=tpe.suggest,
                                      max_evals=100,
                                      trials=trials,
                                      notebook_name='Processing')

In [ ]:
dest_directory = 'model_backup/'
'''
best_model = load_model(dest_directory + 'best_model.bak')

with open(dest_directory+'best_run.pkl', 'rb') as f:  
    best_run = pickle.load(f)    
'''
print("Evalutation of best performing model:")
print(best_model.evaluate(x_test, y_test))
print("Best performing model chosen hyper-parameters:")
print(best_run)
best_model.summary()




now = datetime.datetime.now()
dest_directory_temp =dest_directory + 'bestModel('+now.strftime("%m-%d %H.%M")+")"
if not os.path.exists(dest_directory_temp):
      os.makedirs(dest_directory_temp)
best_model.save(dest_directory_temp + '/best_model.bak')

with open(dest_directory_temp + '/best_run.pkl', 'wb') as f:  
    pickle.dump(best_run, f)



In [ ]:
trials.best_trial